In [ ]:
from time import sleep

def inc(x):
    sleep(1)
    return x + 1

def add(x, y):
    sleep(1)
    return x + y

In [ ]:
%%time

x = inc(1)
y = inc(2)
z = add(x, y)

## Parallelize with the `dask.delayed`

In [ ]:
from dask import delayed

@delayed
def inc_d(x):
    sleep(1)
    return x + 1

@delayed
def add_d(x, y):
    sleep(1)
    return x + y

In [ ]:
%%time

x = inc_d(1)
y = inc_d(2)
z = add_d(x, y)

In [ ]:
# z.visualize()

In [ ]:
%%time

z.compute()

## Parallelizing a `for` loop

In [ ]:
data = [1, 2, 3, 4, 5, 6, 7, 8]

In [ ]:
%%time
results = []
for x in data:
    y = inc(x)
    results.append(y)

total = sum(results)

In [ ]:
%%time
results = []

for x in data:
    y = delayed(inc)(x)
    results.append(y)

total = delayed(sum)(results)

total.compute()

In [ ]:
# total.visualize()

# Dask DataFrame

In [ ]:
# Download the NYC Flights dataset to our workspace
import urllib

print("- Downloading NYC Fligths dataset... ", end='', flush=True)
url = 'https://storage.googleapis.com/dask-tutorial-data/nycflights.tar.gz'
filename, headers = urllib.request.urlretrieve(url, 'nycflights.tar.gz')
print('Done!', flush=True)

In [ ]:
import tarfile

# Extract the .csv files from the tar file
with tarfile.open(filename, mode='r:gz') as flights:
    flights.extractall('data/')

In [ ]:
import os
import dask.dataframe as dd

df = dd.read_csv(os.path.join('data', 'nycflights', '*.csv'), parse_dates={'Date': [0, 1, 2]})

df

In [ ]:
df.head()

In [ ]:
# df.tail()   # ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`

In [ ]:
# For columns with mixed dtypes we have to infer the correct type
df = dd.read_csv(os.path.join('data', 'nycflights', '*.csv'),
                    parse_dates={'Date': [0, 1, 2]},
                    dtype={'TailNum': str, 'CRSElapsedTime': float, 'Cancelled': bool}
)

In [ ]:
df.tail()

## Computations with `dask.dataframe`

In [ ]:
%time df.DepDelay.max().compute()

In [ ]:
# df.DepDelay.max().visualize(rankdir='LR', size='12, 12!')

## Machine Learning with Dask

In [ ]:
# You can grab columns from the Dask DataFrame just as you would with Pandas
df_train = df[['CRSDepTime', 'CRSArrTime', 'Cancelled']]

df_train.shape

In [ ]:
# Perform searches and operations on the data
df_train.isnull().sum().compute()